In [16]:
from prep_data import Data_prepare
import numpy as np 
# import prep_data

###  Data preprocessing 
1. covert mat data to numpy array 
2. combine male and female data to one array 
3. if needed, remove certain rows 
4. get postive and negative of Xs 
    - order difference or net vs neg 
5. reshape each X: 128*8*2 = 2048 
    - input: data 
6. generate labels for x 
    - input: data and label_type (0:zeros, 1: ones )
7. combine net and neg 
    - input: netX,netY,negX,negY

In [9]:
dp = Data_prepare() 
pos = [[0,3],[0,2],[0,1],[0,0]]
neg = [[1,3],[1,2],[1,1],[1,0]]
# netural vs postive 
# pos = [[0,0],[0,1],[1,2],[1,3],[2,0],[2,1],[2,2],[2,3]]
# neg = [[0,2],[0,3],[1,0],[1,1],[3,0],[3,1],[3,2],[3,3]] 
m_file = "./raw_data/Emotrans1_Boy_data_preprocessed_42.mat" 
f_file = "./raw_data/Emotrans1_girl_data_preprocessed_42.mat"
index=[40,41]
df = dp.combine_male_female(m_file,f_file)
df = dp.remove_person(df,index)
pos_df = dp.get_pos_or_neg(df,pos)
neg_df = dp.get_pos_or_neg(df,neg)
pos_sequeezed = dp.squeeze_feature_size(pos_df)
neg_sequeezed = dp.squeeze_feature_size(neg_df)
pos_labels = dp.generate_labels(pos_sequeezed,1)
neg_labels = dp.generate_labels(neg_sequeezed,0)
x,y = dp.combine_net_neg(pos_sequeezed,pos_labels,neg_sequeezed,neg_labels)

(40,)
(18, 128, 8, 2)
(18, 128, 8, 2)
(18, 128, 8, 2)
(18, 128, 8, 2)
(18, 128, 8, 2)
(18, 128, 8, 2)
(18, 128, 8, 2)
(18, 128, 8, 2)
(18, 128, 8, 2)
(18, 128, 8, 2)
(15, 128, 8, 2)
(15, 128, 8, 2)
(18, 128, 8, 2)
(15, 128, 8, 2)
(18, 128, 8, 2)
(18, 128, 8, 2)
(18, 128, 8, 2)
(18, 128, 8, 2)
(12, 128, 8, 2)
(18, 128, 8, 2)
(18, 128, 8, 2)
(18, 128, 8, 2)
(18, 128, 8, 2)
(18, 128, 8, 2)
(18, 128, 8, 2)
(18, 128, 8, 2)
(18, 128, 8, 2)
(18, 128, 8, 2)
(18, 128, 8, 2)
(18, 128, 8, 2)
(18, 128, 8, 2)
(18, 128, 8, 2)
(18, 128, 8, 2)
(18, 128, 8, 2)
(18, 128, 8, 2)
(18, 128, 8, 2)
(18, 128, 8, 2)
(18, 128, 8, 2)
(18, 128, 8, 2)
(15, 128, 8, 2)
(40,)
(18, 128, 8, 2)
(18, 128, 8, 2)
(18, 128, 8, 2)
(18, 128, 8, 2)
(18, 128, 8, 2)
(18, 128, 8, 2)
(18, 128, 8, 2)
(18, 128, 8, 2)
(18, 128, 8, 2)
(18, 128, 8, 2)
(15, 128, 8, 2)
(15, 128, 8, 2)
(18, 128, 8, 2)
(15, 128, 8, 2)
(18, 128, 8, 2)
(18, 128, 8, 2)
(18, 128, 8, 2)
(18, 128, 8, 2)
(12, 128, 8, 2)
(18, 128, 8, 2)
(18, 128, 8, 2)
(18, 128, 8,

In [21]:
## (40,) -> 40*36*2048
def flatten(data):
    res = data[0]
    for i in range(1,data.shape[0]):
        res = np.append(res,data[i],axis=0)
    return res 
new_x = flatten(x)
new_y = flatten(y)

In [22]:
print(new_x.shape,new_y.shape)


(1404, 2048) (1404, 1)
